In [1]:
import random
import math
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
from astropy.io import fits
import numpy as np
import os
import pickle
import keras
%matplotlib inline

Using TensorFlow backend.


In [6]:
%load_ext autoreload
%autoreload 2
from NNutils import create_fname_dict, train_val_split

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
path_to_data ='ALLrescaled'

In [3]:
dfPath = 'path_to_foulder'
path_to_data = 'path_to_data'
df = pd.read_pickle(os.path.join(dfPath,'infoDf.pkl'))

In [4]:
df.head()

instr_type     max      min number_of_ss size class  \
date       region                                                       
1996-06-10 7971          MDI  650.25  -263.82            2   10   BXO   
1996-06-27 7973          MDI  635.91 -2353.96            1   70   HSX   
           7976          MDI  423.33 -1444.87            4   10   AXX   
1996-06-28 7976          MDI  569.68 -1676.75            4   20   HRX   
           7973          MDI  448.23 -2075.18            1   70   HSX   

                  location magn_class letter_1 letter_2 letter_3  
date       region                                                 
1996-06-10 7971     S07W15          B        B        X        O  
1996-06-27 7973     N09W21          A        H        S        X  
           7976     N13E25          A        A        X        X  
1996-06-28 7976     N13E11          A        H        R        X  
           7973     N08W35          A        H        S        X

In [15]:
names_dict, label_encoder = create_fname_dict(df,'letter_1')
val_name_dict, train_name_dict  = train_val_split(names_dict, share = 0.2)

In [175]:
val_name_dict, train_name_dict = train_val_split(names_dict, share = 0.2)

In [174]:
def load_data_by_name_dict(name_dict, shuffle = True):
    
    fnames = []
    Xlist = []
    ylist = []
    for key in name_dict.keys():
        print('Process: '+ key)
        for fname in tqdm(name_dict[key]):
            hdulist = fits.open(fname)
            X =  hdulist[0].data
            y = label_encoder.transform([key])
            y = np.squeeze(keras.utils.to_categorical(y, num_classes=len(val_name_dict.keys())))
            X = X.reshape(1, X.shape[0], X.shape[1]).astype('float32')
            Xlist.append(X)
            ylist.append(y)
            fnames.append(fname)
    
    print('Merge all data to array')
    y = np.array(ylist)
    fnames = np.array(fnames)    
    x_stats = np.array([log_stats(x) for x in Xlist])
        #x_log = np.array([np.sign(x)*np.log1p(np.abs(x)) for x in batch_X])/8.24
    X= np.array(Xlist)/3900
    if shuffle:
        len_ = y.shape[0]
        idx = np.arange(len_)
        np.random.shuffle(idx)
        return ([x_stats[idx], X[idx]], y[idx], fnames[idx])
    else:
        return ([x_stats, X], y,fnames) 
def log_stats(x):
    stats = np.percentile(x,[0.1, 1,5,10,90,95,99,99.9])
    return np.sign(stats)*np.log1p(np.abs(stats))/8.24

## Building neural networks

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, Input, Concatenate
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Activation, BatchNormalization, regularizers
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler
from keras.callbacks import ModelCheckpoint
from keras.layers.pooling import AveragePooling2D
from keras import backend as K
K.set_image_dim_ordering('th')

In [185]:
def conv_model(fsize=5, nb_filters = 64):
    '''Base network to be shared (eq. to feature extraction).
    '''
    input_seq = Input(shape=(1,100,125))
    droprate = 0.3
    #nb_filters  = 150
    convolved = Conv2D(nb_filters, (fsize, fsize), activation="tanh")(input_seq)
    convolved = AveragePooling2D()(convolved)
    convolved  = BatchNormalization()(convolved)
    convolved = Conv2D(nb_filters, (fsize, fsize), activation="tanh")(convolved)
    convolved = AveragePooling2D()(convolved)
    convolved  = BatchNormalization()(convolved)
    convolved = Conv2D(nb_filters, (fsize, fsize), activation="tanh")(convolved)
    convolved = AveragePooling2D()(convolved)
    convolved  = BatchNormalization()(convolved)
    processed = Flatten()(convolved)
    compressed = Dense(50, activation="tanh")(processed)
    #compressed = Dense(50, activation="tanh")(processed)
    compressed = Dropout(droprate)(compressed)
    #compressed = Dense(20, activation="tanh")(compressed)
    model = Model(inputs=input_seq, outputs=compressed)            
    return model
def main_model(fsize=5, nclass = 7, nb_filters = 64):
    input_1 = Input(shape = (8,))
    input_2 = Input(shape=(1,100,125))

    c_model = conv_model(fsize, nb_filters)
    embed1 = Dense(50, activation="tanh")(input_1)
    embed2 = c_model(input_2)

    merged = Concatenate()([embed1, embed2])
    out = Dense(nclass, activation='softmax')(merged)
    model = Model(inputs=[input_1, input_2], outputs=out)
    return model
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))    
def learn_letter1(BS=500, nb_epoch=100):
    loss_history = LossHistory()
    earlyStopping = EarlyStopping(monitor='val_loss', patience=20, verbose=0, mode='auto')
    filepath="letter1-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [loss_history, earlyStopping,checkpoint]
    my_model= main_model()
    my_model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

    my_model.fit(Xtrain, ytrain_let1,batch_size = BS, epochs=nb_epoch, 
                      callbacks=callbacks_list, verbose=1, shuffle = True, validation_data=(Xval,yval_let1))
    return my_model, loss_history 
def learn_letter2(BS=500, nb_epoch=100):
    loss_history = LossHistory()
    earlyStopping = EarlyStopping(monitor='val_loss', patience=20, verbose=0, mode='auto')
    filepath="letter2-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [loss_history, earlyStopping,checkpoint]
    my_model= main_model(nclass = yval_let2.shape[1])
    my_model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])
    BS = 500
    nb_epoch = 100

    my_model.fit(Xtrain, ytrain_let2,batch_size = BS, epochs=nb_epoch, 
                      callbacks=callbacks_list, verbose=1, shuffle = True, validation_data=(Xval,yval_let2))
    return my_model, loss_history
def learn_letter3(BS=500, nb_epoch=100):
    loss_history = LossHistory()
    earlyStopping = EarlyStopping(monitor='val_loss', patience=20, verbose=0, mode='auto')
    filepath="letter3-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [loss_history, earlyStopping,checkpoint]
    my_model= main_model(nclass = yval_let3.shape[1])
    my_model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])
    BS = 500
    nb_epoch = 100

    my_model.fit(Xtrain, ytrain_let3,batch_size = BS, epochs=nb_epoch, 
                      callbacks=callbacks_list, verbose=1, shuffle = True, validation_data=(Xval,yval_let3))
    return my_model, loss_history

In [186]:
let_1_model, loss_history1 = learn_letter1(BS=500, nb_epoch=200)
let_2_model, loss_history2 = learn_letter2(BS=500, nb_epoch=200)
let_3_model, loss_history3 = learn_letter3(BS=500, nb_epoch=200)